 (x, y) から z の高さを求めるコード以下のコードは、指定した (x, y) 座標から真上（$+z$方向）または真下（$-z$方向）にレイを飛ばし、メッシュとの衝突点を計算することで高さを求めます。

In [ ]:
import trimesh
import numpy as np

def get_z_height(mesh, x_coord, y_coord):
    """
    指定された (x, y) 座標におけるメッシュの z の高さを計算します。
    レイキャスティングを使用し、その座標から z 軸方向にレイを飛ばして衝突点を求めます。

    Args:
        mesh (trimesh.Trimesh): ロードされた3Dメッシュオブジェクト。
        x_coord (float): x 座標。
        y_coord (float): y 座標。

    Returns:
        float or None: 衝突した点の z 座標 (高さ)。衝突しなかった場合は None。
    """
    
    # 1. レイキャスターを準備
    # trimesh.ray.ray_pyembree.RayMeshIntersector を使用すると高速です
    # ただし、モデルによってはシンプルな RayMeshIntersector を使用します
    try:
        intersector = trimesh.ray.ray_pyembree.RayMeshIntersector(mesh)
    except:
        intersector = trimesh.ray.RayMeshIntersector(mesh)

    # 2. レイの設定
    # - 原点 (origins): 非常に低いz座標からレイをスタートさせ、モデル全体をカバーするようにします
    # - 方向 (directions): z軸の正の方向 (+z) に向けてレイを飛ばします
    
    # モデルのバウンディングボックスの z_min を取得し、それよりも少し低い位置をレイの開始点とします
    z_min = mesh.bounds[0, 2] - 1.0 
    
    # レイの原点: [x, y, z_min]
    origins = np.array([[x_coord, y_coord, z_min]])
    
    # レイの方向: [0, 0, 1] (z軸の正の方向)
    directions = np.array([[0.0, 0.0, 1.0]])

    # 3. レイの衝突計算
    # locations は、レイがメッシュに衝突した全ての点の座標を返します
    locations, index_ray, index_tri = intersector.intersects_location(
        origins=origins, 
        directions=directions
    )

    if len(locations) > 0:
        # 衝突した点が存在する場合、最も低い z 座標の点（モデルの上面）を返します。
        # ただし、多くの場合、最も近い衝突点（index 0）が求めたい高さです。
        # 単一の閉じたモデルの場合、最初の衝突点が上面であることが一般的です。
        return locations[0, 2] 
    else:
        # 衝突しなかった場合 (モデルの外側)
        return None

# --- 使用例 ---

# 1. 3Dモデルをロード (ファイルのパスを指定してください)
# 例: 'my_model.stl' や 'building.obj' など
try:
    # 実際のモデルファイルパスに置き換えてください
    my_mesh = trimesh.load('my_model.stl') 
except FileNotFoundError:
    # ファイルがない場合のフォールバックとしてサンプルデータを使用
    print("指定されたファイルが見つかりませんでした。サンプルデータ (球体) を使用します。")
    my_mesh = trimesh.creation.icosphere(subdivisions=2, radius=10.0) 

# 2. 高さを求めたい (x, y) 座標を指定
target_x = 0.0
target_y = 0.0

# 3. 高さを計算
height = get_z_height(my_mesh, target_x, target_y)

# 4. 結果の表示
if height is not None:
    print(f"({target_x}, {target_y}) におけるモデルの z の高さ: {height:.3f}")
else:
    print(f"({target_x}, {target_y}) ではモデルと衝突しませんでした。")

### 水平・垂直な面の数を計算するコード

In [ ]:
import trimesh
import numpy as np

def count_horizontal_and_vertical_faces(mesh, angle_tolerance=5.0):
    """
    メッシュの法線ベクトルを分析し、水平面と垂直面の数を計算します。

    Args:
        mesh (trimesh.Trimesh): ロードされた3Dメッシュオブジェクト。
        angle_tolerance (float): 水平/垂直とみなす許容角度 (度)。

    Returns:
        tuple: (水平面の数, 垂直面の数)
    """
    if not isinstance(mesh, trimesh.Trimesh):
        # シーンオブジェクトの場合は最初のメッシュを使用するなど、適切に処理
        print("Trimeshオブジェクトではありません。")
        return 0, 0

    # 1. 各面の法線ベクトルを取得
    # face_normals は (N, 3) の形状で、Nは面の数、3は (nx, ny, nz) の成分
    face_normals = mesh.face_normals

    # 2. 水平な面の基準（Z軸）
    # Z軸方向のベクトル: [0, 0, 1]
    z_axis = np.array([0.0, 0.0, 1.0])

    # 3. 法線ベクトルとZ軸との間の角度を計算
    # trimesh.util.vector_angle を使用して、各法線とZ軸の間の角度 (ラジアン) を一括で計算
    angles_rad = trimesh.util.vector_angle(face_normals, z_axis)
    angles_deg = np.degrees(angles_rad)

    # 4. 水平面の数をカウント
    # 法線がZ軸とほぼ平行 (角度が0°または180°に近い) な面を水平面と見なします。
    # 許容誤差を考慮し、角度が tolerance より小さい、または (180 - tolerance) より大きい場合をカウントします。
    # ここでは、角度が 90° - tolerance から 90° + tolerance の範囲外にある面を探します。
    
    # 0度または180度に近い (水平)
    is_horizontal_positive = angles_deg < angle_tolerance
    is_horizontal_negative = angles_deg > (180.0 - angle_tolerance)
    
    # 論理和で結合
    is_horizontal = is_horizontal_positive | is_horizontal_negative
    num_horizontal = np.sum(is_horizontal)

    # 5. 垂直面の数をカウント
    # 法線がZ軸とほぼ垂直 (角度が90°に近い) な面を垂直面と見なします。
    # 許容誤差を考慮し、角度が (90 - tolerance) から (90 + tolerance) の範囲内にある場合をカウントします。
    
    # 90度に近い (垂直)
    is_vertical = (angles_deg >= (90.0 - angle_tolerance)) & \
                  (angles_deg <= (90.0 + angle_tolerance))
    num_vertical = np.sum(is_vertical)

    return int(num_horizontal), int(num_vertical)

# --- 使用例 ---

# 1. サンプルデータとして立方体を作成
mesh = trimesh.creation.box()

# 2. カウントを実行 (許容角度を5度に設定)
num_horizontal, num_vertical = count_horizontal_and_vertical_faces(mesh, angle_tolerance=5.0)

# 3. 結果の表示
print(f"メッシュの総面数: {len(mesh.faces)}")
print(f"水平な面 (床や天井など): {num_horizontal}") # 立方体なら 2
print(f"垂直な面 (壁など): {num_vertical}")       # 立方体なら 4